In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
PATH = Path('../data/')
TRAIN_PATH = PATH/'train_images'
TEST_PATH_1 = PATH/'leaderboard_test_data'
TEST_PATH_2 = PATH/'leaderboard_holdout_data'

In [ ]:
tfms = get_transforms(flip_vert=True, max_lighting=0.4, max_zoom=1.05, max_warp=0.,
                      p_affine=1., p_lighting=0.)

In [ ]:
np.random.seed(42)

src = (ImageItemList.from_csv(PATH, 'traininglabels.csv', folder='train_images')
                    .random_split_by_pct(0.2)
                    .label_from_df(cols=1))

In [ ]:
bs = 16
data = (src.transform(tfms, size=128)
           .databunch(bs=bs)
           .normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(12,9))

In [ ]:
arch = models.resnet34

In [ ]:
learn = create_cnn(data, arch, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 1e-2

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-rn50')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find(start_lr=1e-7, end_lr=1e-1)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(1e-4, lr/5))

In [ ]:
learn.save('stage-2-rn50')

In [ ]:
data = (src.transform(tfms, size=256)
           .databunch(bs=64)
           .normalize(imagenet_stats))
learn.data = data

In [ ]:
learn.freeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr=1e-2/2

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-256-rn50')

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('stage-2-256-rn50')

In [ ]:
learn.export()

# Test

In [ ]:
test_1 = ImageItemList.from_folder(TEST_PATH_1)
test_2 = ImageItemList.from_folder(TEST_PATH_2)
len(test_1), len(test_2)

In [ ]:
learn_1 = load_learner(PATH, test=test_1)
preds_1, _ = learn_1.get_preds(ds_type=DatasetType.Test)
_, out_1 = torch.max(preds_1, 1)

In [ ]:
fnames_1 = [str(f).split('/')[-1] for f in learn_1.data.test_ds.items]
df_1 = pd.DataFrame({'image_id':fnames_1, 'has_oilpalm':out_1}, columns=['image_id', 'has_oilpalm'])

In [ ]:
df_1.head(5)

In [ ]:
learn_2 = load_learner(PATH, test=test_2)
preds_2, _ = learn_2.get_preds(ds_type=DatasetType.Test)
_, out_2 = torch.max(preds_2, 1)

In [ ]:
fnames_2 = [str(f).split('/')[-1] for f in learn_2.data.test_ds.items]
df_2 = pd.DataFrame({'image_id':fnames_2, 'has_oilpalm':out_2}, columns=['image_id', 'has_oilpalm'])

In [ ]:
df_2.head(5)

In [ ]:
df_out = pd.concat([df_1, df_2], axis=0).sort_values('image_id').reset_index(drop=True)

In [ ]:
df_out.shape

In [ ]:
df_out.head(10)

In [ ]:
df_out.to_csv('./test_output.csv', index=False)